In [5]:
################################################################################################################################################################################
# Importamos las librerías necesarias y generamos conexión con Binance para descargar la lista de pares que binance maneja.
################################################################################################################################################################################
import pandas as pd
import os
import numpy as np
from datetime import datetime
import sympy as sp
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from binance import Client
from time import sleep
from IPython.display import clear_output

client = Client()

##################################################
# Lista de parámetros.
##################################################
# Parámetros de interpolación.
###################################
smooth_interval = 10
smooth_exp = 3
###################################
# Parámetros de estrategia.
###################################
rango = 0
std_mult = 0.25
pdi_mult = 500
sdi_mult = 100000
z_aprox = 0.00005
pmax_idx = 0
pmin_idx = 0

def savgol_filter(y, window_size, poly_order):
    half_window = window_size // 2
    order_range = np.arange(poly_order+1)
    weight = np.zeros((window_size, poly_order+1))
    
    for i in range(-half_window, half_window):
        weight[i+half_window, :] = [i**j for j in order_range]
    
    weights = np.linalg.pinv(weight).T.sum(axis=0)
    
    smoothed = np.convolve(weights, y, mode='valid')
    
    return smoothed #smoothed_padded

def calculate_derivatives(interpolacion, times, derivative):
    d_interpolacion = np.diff(interpolacion) / times[2+derivative:]
    return d_interpolacion


# while True:
#     clear_output(wait=True)
##################################################################################################################################
# Descarga y limpieza de datos de X días atrás a hoy.
##################################################################################################################################
recent_data = pd.read_csv('../../../csvs/streaming/BTCUSDT.csv')
recent_data.columns = ['Tiempo','Simbolo','Open']
recent_data = recent_data.loc[:,['Tiempo','Open']]
recent_data['Tiempo'] = recent_data.Tiempo.astype('datetime64')
recent_data = recent_data.set_index('Tiempo')
recent_data['Open'] = recent_data.Open.astype(float)
recent_data = recent_data.reset_index()

##################################################
# Interpolación (suavizado)  del Precio (OPEN) con SciPy y obtención de derivadas con su respectiva interpolación.
##################################################
open_prices = recent_data['Open'].values
times = (np.diff(recent_data.Tiempo)/np.timedelta64(1, 's')).astype(float)

interpolacion = savgol_filter(open_prices, smooth_interval, 3)
primer_derivada = calculate_derivatives(interpolacion, times, 1)
pd_interpolacion = savgol_filter(primer_derivada, smooth_interval, 3)
segunda_derivada = calculate_derivatives(pd_interpolacion, times, 5)
sd_interpolacion = savgol_filter(segunda_derivada, smooth_interval, 3)
tercer_derivada = calculate_derivatives(sd_interpolacion, times, 9)
td_interpolacion = savgol_filter(tercer_derivada, smooth_interval, 3)
recent_data['Primer_Derivada'] = np.concatenate([np.zeros(3),interpolacion])
recent_data['Primer_Derivada'] = np.concatenate([np.zeros(4),primer_derivada])
recent_data['PD_Interpolacion'] = np.concatenate([np.zeros(7),pd_interpolacion])
recent_data['Segunda_Derivada'] = np.concatenate([np.zeros(8),segunda_derivada])
recent_data['SD_Interpolacion'] = np.concatenate([np.zeros(11),sd_interpolacion])
recent_data['Tercer_Derivada'] = np.concatenate([np.zeros(12),tercer_derivada])
recent_data['TD_Interpolacion'] = np.concatenate([np.zeros(15),td_interpolacion])

######################################################################
# Agregamos el código necesario para encontrar los puntos exactos mínimos y máximos
######################################################################
initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[:1].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() + rango
final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo[-1:].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() - rango

lista_min_max = recent_data[initial_date_index:final_date_index][(recent_data[initial_date_index:final_date_index].TD_Interpolacion >= -z_aprox) & (recent_data[initial_date_index:final_date_index].TD_Interpolacion <= z_aprox)]
indices_min_max = lista_min_max.index.tolist()

max_points_idx = []
min_points_idx = []
pmin = 0
pmax = 0

recent_data['sdi_updown'] = np.select(condlist=[(abs(recent_data['SD_Interpolacion']*100) > abs(np.std(recent_data.Open)*std_mult))],
                                            choicelist=[1],
                                            default=0)
recent_data['pdi_updown'] = np.select(condlist=[((recent_data.PD_Interpolacion*pdi_mult) > abs(np.std(recent_data.Open)*std_mult)),
                                                ((recent_data.PD_Interpolacion*pdi_mult) < -abs(np.std(recent_data.Open)*std_mult))],
                                                choicelist=[1,-1],
                                                default=0)
recent_data['sell_buy_side'] = np.select(condlist=[recent_data['SD_Interpolacion'] > 0,recent_data['SD_Interpolacion'] < 0],
                                        choicelist=[1,-1],
                                        default=0)

if (recent_data['sell_buy_side'] == 1) & (recent_data['sdi_updown'] == 1) & (recent_data['pdi_updown'] == 1):
        max_points_idx.append(pmax_idx)
elif (recent_data['sell_buy_side'] == -1) & (recent_data['sdi_updown'] == 1) & (recent_data['pdi_updown'] == -1):
        min_points_idx.append(pmin_idx)
            
max_points_idx = list(set(max_points_idx))
min_points_idx = list(set(min_points_idx))
max_points_idx.sort()
min_points_idx.sort()

######################################################################
# Ahora ya podemos visualizar los resultados con el siguiente código.
######################################################################
mp = recent_data.Open[initial_date_index:final_date_index].mean()
plt.figure(figsize=(15,8))
plt.plot(recent_data.Tiempo[initial_date_index - rango:final_date_index + rango],recent_data.Open[initial_date_index - rango:final_date_index + rango].multiply(1),color='y')
plt.plot(recent_data.Tiempo[initial_date_index:final_date_index],recent_data.PD_Interpolacion[initial_date_index:final_date_index].multiply(pdi_mult)+mp,'purple')
plt.plot(recent_data.Tiempo[initial_date_index:final_date_index],recent_data.SD_Interpolacion[initial_date_index:final_date_index].multiply(sdi_mult)+mp,color='k')
# plt.plot(recent_data.Tiempo[initial_date_index:final_date_index],recent_data.TD_Interpolacion[initial_date_index:final_date_index].multiply(300)+mp,color='y')
plt.scatter(recent_data.Tiempo.iloc[max_points_idx],recent_data.Open.iloc[max_points_idx],color='g')
plt.scatter(recent_data.Tiempo.iloc[min_points_idx],recent_data.Open.iloc[min_points_idx],color='r')
plt.grid(visible=True)

# Set minor tick locations.
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%y %H:%M'))
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=10))
plt.gca().yaxis.set_major_locator(ticker.LinearLocator(40))
plt.gcf().autofmt_xdate()

# Set grid to use minor tick locations. 
plt.grid(which = 'minor')
plt.xticks(rotation=75)
plt.axhline(mp)
plt.axhline(np.std(recent_data.Interpolacion)*std_mult + mp)
plt.axhline(mp - np.std(recent_data.Interpolacion)*std_mult)
# plt.axvline(datetime.strptime('2024-04-04 09:59:00','%Y-%m-%d %H:%M:%S'))
plt.show()
# sleep(15)

AttributeError: 'DataFrame' object has no attribute 'TD_Interpolacion'